In [1]:
# Import libraries
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import json
import traceback
import numpy as np

fablib = fablib_manager()
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Project ID,0508639f-d515-44c8-b922-cf4cf000b6d9
Token File,/home/fabric/.tokens.json
Bastion Host,bastion.fabric-testbed.net
Bastion Username,bschlue_0000009427
Bastion Private Key File,/home/fabric/work/fabric_config/bastion-key
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Log File,/tmp/fablib/fablib.log
Log Level,INFO


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Project ID,0508639f-d515-44c8-b922-cf4cf000b6d9
Token File,/home/fabric/.tokens.json
Bastion Host,bastion.fabric-testbed.net
Bastion Username,bschlue_0000009427
Bastion Private Key File,/home/fabric/work/fabric_config/bastion-key
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Log File,/tmp/fablib/fablib.log
Log Level,INFO


In [2]:
# Setup constants
cores = 1
ram = 2
disk = 10
nBots = 1
nUsers = 1
nNodes = 2 + nBots + nUsers

In [12]:
# Define and Submit Slice
slice_name = "ECE6400-Final-DDoS-Sim"
site = "EDUKY"
print(site)

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

# Nodes 
#userNode
#controllerNode
#targetNode
#botNodes = [None] * nBots
#userNodes = [None] * nUsers

# Interfaces
#userInterface
#controllerInterfaces
#targetInterfaces
#botInterfaces
#userInterfaces

try:
    slice = fablib.new_slice(name=slice_name)
    
    # Create all of the nodes for the network
    userNode = slice.add_node(name="User", site=site)
    userNode.set_capacities(cores=cores, ram=ram, disk=disk)
    userNode.set_image(image)    
    user_ap = userNode.add_component(model=nicmodel, name="user-nic").get_interfaces()[0]
    
    targetNode = slice.add_node(name="Target", site=site)
    targetNode.set_capacities(cores=cores, ram=ram, disk=disk)
    targetNode.set_image(image)    
    target_ap1 = targetNode.add_component(model=nicmodel, name="target-nic1").get_interfaces()[0] 
    target_ap2 = targetNode.add_component(model=nicmodel, name="target-nic2").get_interfaces()[0] 
    
    # 1 bot for now while gettings things up and running
    botNode = slice.add_node(name="Bot", site=site)
    botNode.set_capacities(cores=cores, ram=ram, disk=disk)
    botNode.set_image(image)    
    bot_ap1 = botNode.add_component(model=nicmodel, name="bot-nic1").get_interfaces()[0]
    bot_ap2 = botNode.add_component(model=nicmodel, name="bot-nic2").get_interfaces()[0]
    
    controllerNode = slice.add_node(name="Controller", site=site)
    controllerNode.set_capacities(cores=cores, ram=ram, disk=disk)
    controllerNode.set_image(image)    
    controller_ap = controllerNode.add_component(model=nicmodel, name="controller-nic").get_interfaces()[0] 
    
    '''
    for i in range(0, nBots):
        botNodes[i] = slice.add_node(name="Bot_" + str(i), site=site)
        botNodes[i].set_capacities(cores=cores, ram=ram, disk=disk)
        botNodes[i].set_image(image)    
        Ap1 = botNodes[i].add_component(model=nicmodel, name="bot-nic-" + str(i)).get_interfaces()[0] 
        Ap2 = botNodes[i].add_component(model=nicmodel, name="bot-nic-" + str(i)).get_interfaces()[0]   
    '''
    
    # Network
    link_target_user = slice.add_l2network(name="tu_bridge", interfaces=[target_ap1, user_ap])
    link_bot_target = slice.add_l2network(name="bt_bridge", interfaces=[bot_ap1, target_ap2])
    link_controller_bot = slice.add_l2network(name="cb_bridge", interfaces=[controller_ap, bot_ap2])
    
    #Submit Slice Request
    slice.submit()

except Exception as e:
    print(f"Slice Failed: {e}")


Retry: 11, Time: 286 sec


ID,b0ec2c92-3b43-465c-9406-d03af4c91338
Name,ECE6400-Final-DDoS-Sim
Lease Expiration (UTC),2025-04-01 00:25:44 +0000
Lease Start (UTC),2025-03-31 00:25:44 +0000
Project ID,0508639f-d515-44c8-b922-cf4cf000b6d9
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
bb567104-7f17-45b2-b2e2-430935cb2664,Bot,1,2,10,default_ubuntu_20,qcow2,eduky-w17.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:feae:1aa0,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:feae:1aa0,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
f15703c9-204e-4fed-b314-79a5543c92ab,Controller,1,2,10,default_ubuntu_20,qcow2,eduky-w6.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe34:2a9d,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe34:2a9d,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
30a9c763-c5c7-4381-9e17-4e8bc4ff4085,Target,1,2,10,default_ubuntu_20,qcow2,eduky-w13.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe18:d0f8,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe18:d0f8,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
0bca4124-b094-49c6-bc3a-48a97786d0b6,User,1,2,10,default_ubuntu_20,qcow2,eduky-w15.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fee6:a324,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fee6:a324,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
bb3bb0c3-bf93-41b5-8b54-f2f470d25bdb,bt_bridge,L2,L2Bridge,EDUKY,None,None,Active,
e38d324f-5cd8-4f5f-958f-1fb6a11dd073,cb_bridge,L2,L2Bridge,EDUKY,None,None,Active,
ab0e27b1-eb4c-4b81-af9a-18b985c423f7,tu_bridge,L2,L2Bridge,EDUKY,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
User-user-nic-p1,p1,User,tu_bridge,100,config,,02:05:D6:28:EE:72,enp7s0,enp7s0,fe80::5:d6ff:fe28:ee72,1,HundredGigE0/0/0/15
Target-target-nic1-p1,p1,Target,tu_bridge,100,config,,06:A8:BC:8D:42:82,enp8s0,enp8s0,fe80::4a8:bcff:fe8d:4282,1,HundredGigE0/0/0/31
Target-target-nic2-p1,p1,Target,bt_bridge,100,config,,06:4B:E9:E2:1D:0E,enp7s0,enp7s0,fe80::44b:e9ff:fee2:1d0e,1,HundredGigE0/0/0/31
Bot-bot-nic1-p1,p1,Bot,bt_bridge,100,config,,02:DB:0B:5B:3B:D8,enp7s0,enp7s0,fe80::db:bff:fe5b:3bd8,1,HundredGigE0/0/0/35
Bot-bot-nic2-p1,p1,Bot,cb_bridge,100,config,,0E:96:8B:63:8C:C1,enp8s0,enp8s0,fe80::c96:8bff:fe63:8cc1,1,HundredGigE0/0/0/35
Controller-controller-nic-p1,p1,Controller,cb_bridge,100,config,,02:5A:9F:4B:87:1F,enp7s0,enp7s0,fe80::5a:9fff:fe4b:871f,1,HundredGigE0/0/0/24



Time to print interfaces 301 seconds


In [ ]:
client_name = "client"
server_name = "server"
network_service_name = "bridge1"

client_ip = '10.20.30.40'
server_ip = '10.20.30.41'

client = slice.get_node(name=client_name)        
iface1 = client.get_interface(network_name=network_service_name)  
iface1.set_ip(ip=client_ip, cidr="24")

iface1.get_device_name()

client.execute("echo 100.20.30.40 client | sudo tee -a /etc/hosts")
client.execute(f"sudo ip link set dev { iface1.get_device_name()} up")

stdout, stderr = client.execute(f'ip addr show {iface1.get_os_interface()}')
server = slice.get_node(name=server_name)        
iface2 = server.get_interface(network_name=network_service_name)  
iface2.set_ip(ip=server_ip, cidr="24")

stdout, stderr = server.execute(f'ip addr show {iface2.get_os_interface()}')
server.execute(f"sudo ip link set dev { iface2.get_device_name()} up")

print("Printing Server's Network")
out, err = server.execute("ip a ")

print("Getting control interface ip")
out, err = server.execute("ip a | grep inet | grep enp | awk '{print $2}'")
ip = ((out.split('\n')[0]).split('/'))[0]
print(f"Ip found: {ip}")

print("Getting Client's interface name")
print(f'Device Found: {iface1.get_device_name()}')

# add route if needed
client.execute(f"sudo ip route add {ip} dev {iface1.get_device_name()}")